## Howework 7

Напишите декоратор, оптимизирующий работу декорируемой функции. Декоратор должен сохранять результат работы функции на ближайшие три запуска и вместо выполнения функции возвращать сохранённый результат.
После трёх запусков функция должна вызываться вновь, а результат работы функции — вновь кешироваться.

## Реализация
Одна и та же функция может вызываться с разными аргументами, следовательно нужно кешировать результат в зависимости от аргументов функции
для этого буду хранить результат в словаре где ключ - аргументы функции, а значение это результат
через такой подход я смогу возвращать закешированное значение в зависимости от аргументов функции

In [1]:
# Хранение результатов и количество вызовов для определенных аргументов функции
# Хранение происходит в словаре где ключ - аргументы функции преобразованные в строку, значение результат и счетчик вызовов
class CacheStorage:

    # attempt_number максимальное количество вызовов при котором значение выселяется из кеша
    def __init__(self, attempt_number) -> None:
        self.attempt_number = attempt_number
        self.cache = dict()

    # проверяет по списку аргументов есть ли для них результат в кеше
    def has(self, *args, **kwargs):
        key = CacheStorage.key(args, kwargs)
        return key in self.cache and self.cache.get(key).counter < self.attempt_number

    # получает по списку аргументов значение
    def get(self, *args, **kwargs):
        key = CacheStorage.key(args, kwargs)
        return self.cache.get(key)

    # добавляет в кеш значения результата по списку аргументов
    def add(self, result, *args, **kwargs):
        key = CacheStorage.key(args, kwargs)
        self.cache[key] = CachedResult(result)

    # приводит список аргументов в строку
    @staticmethod
    def key(*args, **kwargs):
        return str(args + tuple(kwargs.items()))

# Хранит результат и счетчик вызовов
class CachedResult:

    def __init__(self, result) -> None:
        self.__result = result
        self.counter = 0

    # метод для получения результат. Возвращает результат и увеличивает счетчик вызово на 1
    def get_result(self):
        self.counter += 1
        return self.__result

    def __str__(self) -> str:
        return f"result: [{self.__result}] counter: {self.counter}"

## Декоратор для кеширования

In [2]:
# Кеширует значение декорируемой функции, возвращает закешированный результат 3 раза,
# на четвертый закешированное значение выселяется и добавляется новый результат
def cache(func):
    def wrapper(*args, **kwargs):
        if wrapper.cache.has(args, kwargs):
            cached_result = wrapper.cache.get(args, kwargs)
            print(f'Function: {func.__name__!r} with args: {CacheStorage.key(args, kwargs)} return cached {cached_result}')
            return cached_result.get_result()
        else:
            result = func(*args, **kwargs)
            print(f'Function: {func.__name__!r} with args: {CacheStorage.key(args, kwargs)} return [{result}] and add result to cache')
            wrapper.cache.add(result, args, kwargs)
            return result
    wrapper.cache = CacheStorage(3)
    return wrapper

### Тестовые функции

In [3]:
@cache
def sqr(x):
    return x * x

@cache
def sqrt(x):
    return x ** 0.5

@cache
def perimeter(*args):
    result = 0
    for arg in args:
        result += arg
    return result

@cache
def join(delimeter, *args):
    result = ''
    delimeter = str(delimeter)
    for arg in args:
        result += str(arg) + delimeter
    return result[:-len(delimeter)]

import random
@cache
def random_in_range(start, end):
    return random.randint(start, end)


# Тесты

### Первый вызовов функций добавляет в кеш

In [4]:
sqr(2)
sqr(10)
sqrt(256)
perimeter(1,2,3,4,5)
perimeter(10, 20, 30, 40)
join(' : ', 'Hello', 'World')
join('+', 2, 4, 8, 16)
random_in_range(0, 100)
random_in_range(1000, 9999)

Function: 'sqr' with args: ((2,), {}) return [4] and add result to cache
Function: 'sqr' with args: ((10,), {}) return [100] and add result to cache
Function: 'sqrt' with args: ((256,), {}) return [16.0] and add result to cache
Function: 'perimeter' with args: ((1, 2, 3, 4, 5), {}) return [15] and add result to cache
Function: 'perimeter' with args: ((10, 20, 30, 40), {}) return [100] and add result to cache
Function: 'join' with args: ((' : ', 'Hello', 'World'), {}) return [Hello : World] and add result to cache
Function: 'join' with args: (('+', 2, 4, 8, 16), {}) return [2+4+8+16] and add result to cache
Function: 'random_in_range' with args: ((0, 100), {}) return [28] and add result to cache
Function: 'random_in_range' with args: ((1000, 9999), {}) return [1639] and add result to cache


1639

### Следующие три вызова возвращают результаты из кеша

In [5]:
print("\n----------------- attempt 1 ----------------------\n")
sqr(2)
sqr(10)
sqrt(256)
perimeter(1,2,3,4,5)
perimeter(10, 20, 30, 40)
join(' : ', 'Hello', 'World')
join('+', 2, 4, 8, 16)
random_in_range(0, 100)
random_in_range(1000, 9999)

print("\n----------------- attempt 2 ----------------------\n")

sqr(2)
sqr(10)
sqrt(256)
perimeter(1,2,3,4,5)
perimeter(10, 20, 30, 40)
join(' : ', 'Hello', 'World')
join('+', 2, 4, 8, 16)
random_in_range(0, 100)
random_in_range(1000, 9999)

print("\n----------------- attempt 3 ----------------------\n")
sqr(2)
sqr(10)
sqrt(256)
perimeter(1,2,3,4,5)
perimeter(10, 20, 30, 40)
join(' : ', 'Hello', 'World')
join('+', 2, 4, 8, 16)
random_in_range(0, 100)
random_in_range(1000, 9999)


----------------- attempt 1 ----------------------

Function: 'sqr' with args: ((2,), {}) return cached result: [4] counter: 0
Function: 'sqr' with args: ((10,), {}) return cached result: [100] counter: 0
Function: 'sqrt' with args: ((256,), {}) return cached result: [16.0] counter: 0
Function: 'perimeter' with args: ((1, 2, 3, 4, 5), {}) return cached result: [15] counter: 0
Function: 'perimeter' with args: ((10, 20, 30, 40), {}) return cached result: [100] counter: 0
Function: 'join' with args: ((' : ', 'Hello', 'World'), {}) return cached result: [Hello : World] counter: 0
Function: 'join' with args: (('+', 2, 4, 8, 16), {}) return cached result: [2+4+8+16] counter: 0
Function: 'random_in_range' with args: ((0, 100), {}) return cached result: [28] counter: 0
Function: 'random_in_range' with args: ((1000, 9999), {}) return cached result: [1639] counter: 0

----------------- attempt 2 ----------------------

Function: 'sqr' with args: ((2,), {}) return cached result: [4] counter: 1
F

1639

### На четвертый вызов значения выселяются из кеша и добавляются новые результаты

In [6]:
sqr(2)
sqr(10)
sqrt(256)
perimeter(1,2,3,4,5)
perimeter(10, 20, 30, 40)
join(' : ', 'Hello', 'World')
join('+', 2, 4, 8, 16)
random_in_range(0, 100)
random_in_range(1000, 9999)

Function: 'sqr' with args: ((2,), {}) return [4] and add result to cache
Function: 'sqr' with args: ((10,), {}) return [100] and add result to cache
Function: 'sqrt' with args: ((256,), {}) return [16.0] and add result to cache
Function: 'perimeter' with args: ((1, 2, 3, 4, 5), {}) return [15] and add result to cache
Function: 'perimeter' with args: ((10, 20, 30, 40), {}) return [100] and add result to cache
Function: 'join' with args: ((' : ', 'Hello', 'World'), {}) return [Hello : World] and add result to cache
Function: 'join' with args: (('+', 2, 4, 8, 16), {}) return [2+4+8+16] and add result to cache
Function: 'random_in_range' with args: ((0, 100), {}) return [51] and add result to cache
Function: 'random_in_range' with args: ((1000, 9999), {}) return [2510] and add result to cache


2510